In [11]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer


In [12]:
engine = create_engine("mssql+pyodbc://LAPTOP-E9022B9F\SQLEXPRESS/PortfolioProject_MarketingAnalytics?trusted_connection=yes&driver=SQL+Server")


In [13]:
# Function to fetch data from SQL Server
def fetch_data_from_sql():
    query = "SELECT ReviewID, CustomerID, ProductID, ReviewDate, Rating, ReviewText FROM dbo.customer_reviews"
    df = pd.read_sql(query, engine)
    return df

In [14]:
# Fetch customer reviews
customer_reviews_df = fetch_data_from_sql()

In [15]:
# Initialize Sentiment Analyzer
sia = SentimentIntensityAnalyzer()


In [16]:
# Function to calculate sentiment score
def calculate_sentiment(review):
    sentiment = sia.polarity_scores(str(review))  # Ensure review is a string
    return sentiment['compound']


In [17]:
# Define a function to categorize sentiment using both the sentiment score and the review rating
def categorize_sentiment(score, rating):
    # Use both the text sentiment score and the numerical rating to determine sentiment category
    if score > 0.05:  # Positive sentiment score
        if rating >= 4:
            return 'Positive'  # High rating and positive sentiment
        elif rating == 3:
            return 'Mixed Positive'  # Neutral rating but positive sentiment
        else:
            return 'Mixed Negative'  # Low rating but positive sentiment
    elif score < -0.05:  # Negative sentiment score
        if rating <= 2:
            return 'Negative'  # Low rating and negative sentiment
        elif rating == 3:
            return 'Mixed Negative'  # Neutral rating but negative sentiment
        else:
            return 'Mixed Positive'  # High rating but negative sentiment
    else:  # Neutral sentiment score
        if rating >= 4:
            return 'Positive'  # High rating with neutral sentiment
        elif rating <= 2:
            return 'Negative'  # Low rating with neutral sentiment
        else:
            return 'Neutral'  # Neutral rating and neutral sentiment

In [18]:
# Define a function to bucket sentiment scores into text ranges
def sentiment_bucket(score):
    if score >= 0.5:
        return '0.5 to 1.0'  # Strongly positive sentiment
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'  # Mildly positive sentiment
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0'  # Mildly negative sentiment
    else:
        return '-1.0 to -0.5'  # Strongly negative sentiment

In [19]:
# Apply sentiment analysis
customer_reviews_df['SentimentScore'] = customer_reviews_df['ReviewText'].apply(calculate_sentiment)

In [20]:
# Apply sentiment categorization using both text and rating
customer_reviews_df['SentimentCategory'] = customer_reviews_df.apply(
    lambda row: categorize_sentiment(row['SentimentScore'], row['Rating']), axis=1)

In [32]:
# Apply sentiment bucketing to categorize scores into defined ranges
customer_reviews_df['SentimentBucket'] = customer_reviews_df['SentimentScore'].apply(sentiment_bucket)

In [34]:

# Display the first few rows of the DataFrame with sentiment scores, categories, and buckets
print(customer_reviews_df.head())

   ReviewID  CustomerID  ProductID  ReviewDate  Rating  \
0         1          77         18  2023-12-23       3   
1         2          80         19  2024-12-25       5   
2         3          50         13  2025-01-26       4   
3         4          78         15  2025-04-21       3   
4         5          64          2  2023-07-16       3   

                                 ReviewText  SentimentScore SentimentCategory  \
0   Average  experience,  nothing  special.         -0.3089    Mixed Negative   
1            The  quality  is    top-notch.          0.0000          Positive   
2   Five  stars  for  the  quick  delivery.          0.0000          Positive   
3  Good  quality,  but  could  be  cheaper.          0.2382    Mixed Positive   
4   Average  experience,  nothing  special.         -0.3089    Mixed Negative   

  SentimentBucket  
0    -0.49 to 0.0  
1     0.0 to 0.49  
2     0.0 to 0.49  
3     0.0 to 0.49  
4    -0.49 to 0.0  


In [36]:
# Save the DataFrame with sentiment scores, categories, and buckets to a new CSV file
customer_reviews_df.to_csv('fact_customer_reviews_with_sentiment.csv', index=False)

In [ ]:
import flask
from flask import send_file

app = flask.Flask(__name__)

@app.route('/download')
def download_file():
    return send_file("customer_reviews_sentiment.csv", as_attachment=True)

app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
